In [1]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
# from fastai.column_data import *
# from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.torch_imports import save_model, load_model

In [4]:
from models import *
from utils import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
ls -f KITTI/ | grep .csv

validation.csv
validation_192_640.csv
training.csv
validation_192_640_pre.csv
training_192_640_pre.csv
training_192_640.csv


In [6]:
train, val = pd.read_csv('./KITTI/training_192_640_pre.csv'), pd.read_csv('./KITTI/validation_192_640_pre.csv')
train, val = shuffle(train), shuffle(val);

In [7]:
train.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_30/2011_09_30_drive_0027_sync/image_03...,2011_09_30/2011_09_30_drive_0027_sync/image_03...,2011_09_30/2011_09_30_drive_0027_sync/image_03...,364.362615,362.030694,310.151266,93.752525
1,2011_09_30/2011_09_30_drive_0033_sync/image_02...,2011_09_30/2011_09_30_drive_0033_sync/image_02...,2011_09_30/2011_09_30_drive_0033_sync/image_02...,364.362615,362.030694,310.151266,93.752525
2,2011_10_03/2011_10_03_drive_0042_sync/image_02...,2011_10_03/2011_10_03_drive_0042_sync/image_02...,2011_10_03/2011_10_03_drive_0042_sync/image_02...,370.424992,368.054272,312.885179,94.830438
3,2011_10_03/2011_10_03_drive_0027_sync/image_03...,2011_10_03/2011_10_03_drive_0027_sync/image_03...,2011_10_03/2011_10_03_drive_0027_sync/image_03...,370.424992,368.054272,312.885179,94.830438
4,2011_10_03/2011_10_03_drive_0042_sync/image_02...,2011_10_03/2011_10_03_drive_0042_sync/image_02...,2011_10_03/2011_10_03_drive_0042_sync/image_02...,370.424992,368.054272,312.885179,94.830438


In [8]:
val.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,370.424992,368.054272,312.885179,94.830438
1,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,370.424992,368.054272,312.885179,94.830438
2,2011_09_26/2011_09_26_drive_0022_sync/image_02...,2011_09_26/2011_09_26_drive_0022_sync/image_02...,2011_09_26/2011_09_26_drive_0022_sync/image_02...,371.806866,369.427302,314.104631,88.501248
3,2011_09_26/2011_09_26_drive_0087_sync/image_03...,2011_09_26/2011_09_26_drive_0087_sync/image_03...,2011_09_26/2011_09_26_drive_0087_sync/image_03...,371.806866,369.427302,314.104631,88.501248
4,2011_09_30/2011_09_30_drive_0020_sync/image_02...,2011_09_30/2011_09_30_drive_0020_sync/image_02...,2011_09_30/2011_09_30_drive_0020_sync/image_02...,364.362615,362.030694,310.151266,93.752525


In [9]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [10]:
trn_camera.head()

,fx,fy,cx,cy
0,364.362615,362.030694,310.151266,93.752525
1,364.362615,362.030694,310.151266,93.752525
2,370.424992,368.054272,312.885179,94.830438
3,370.424992,368.054272,312.885179,94.830438
4,370.424992,368.054272,312.885179,94.830438


In [11]:
arch = f
bs = 4
PATH = 'Fastai_TRN'
DPATH = "KITTI/"
verbose = 500

In [12]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [13]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [14]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)

In [15]:
tfms = (trn_tfms, val_tfms)

MD1 = get_MD(trn0, val0, tfms, bs, DPATH, PATH)
MD2 = get_MD(trn1, val1, tfms, bs, DPATH, PATH)
MD3 = get_MD(trn2, val2, tfms, bs, DPATH, PATH)
MDcam = get_cam(trn_camera, val_camera, bs)

In [16]:
imgnet_mean, imgnet_std = torch.from_numpy(stats[0]).float(), torch.from_numpy(stats[1]).float()
imgnet_mean, imgnet_std = imgnet_mean.view(1,3,1,1), imgnet_std.view(1,3,1,1)
denorm = denormer(imgnet_mean, imgnet_std)

In [17]:
class Loss(nn.Module):
    def __init__(self, scale=10, Tscale=2, ndown=2):
        super().__init__()
        self.appr = TriAppearanceLoss(scale=Tscale) #, ndown=ndown)
        self.smooth = SmoothLoss()
        self.scale = scale
    def forward(self, d1s, d2s ,d3s, trans, rotation, x1, x2, x3, camera):
        appr_loss, details = self.appr(d2s, trans, rotation, x1, x2, x3, camera)
        smooth_losses = [ 0.5/(i+1) * self.smooth(d2) for i, d2 in enumerate(d2s) ]
        #smooth_losses = [ self.smooth(d1)+self.smooth(d2) + self.smooth(d3) for d1, d2, d3 in zip(d1s, d2s, d3s) ]
        smooth_loss = torch.mean(torch.cat(smooth_losses, dim=0)) * self.scale
        #print(type(appr_loss))
        #print(type(smooth_loss))
        return appr_loss + smooth_loss, (appr_loss, smooth_loss, *details) 
               

In [18]:
def train(m, opt, MD1, MD2, MD3, MDcam, denorm):
    m.train()
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []  
    
    #for i in range(2): # just for testing
    for i in range(len(MD1.trn_ds)//bs-len(losses)):
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

        d1, d2, d3, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 
        #pdb.set_trace()
        loss, details = l(d1, d2, d3, trans, rotation, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
        # appr smooth ssim l1
        if i%verbose == 0: print(loss.data[0],
                                 details[0].data[0],
                                 details[1].data[0],
                                 details[2].data[0],
                                 details[3].data[0],
                                 sep='\t')
    return losses

In [19]:
def evaluate(m, MD1, MD2, MD3, MDcam, folder):
    
    DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
    m.eval()
    for i in range(3): 
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
        d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1, volatile=True), denorm(imgs2, volatile=True), denorm(imgs3, volatile=True) 
        #pdb.set_trace()
        d2 = d2s[0]
        # d2 = F.upsample(input=d2, scale_factor=2**di, mode='bilinear')
        cx12, cy12, dm12 = l.appr.offset.forward(trans=trans[:,0], rotation=rotation[:,0], inv_depth = d2, camera = cam)
        #pdb.set_trace()

        x12, ivm12 = l.appr.sampler.forward(imgs1, cx12, cy12)
        ivm12.volatile = True

        del dm12 
        del ivm12
        
        imgs2 = tonp(imgs2.permute(0,2,3,1))
        x12 = tonp(x12.permute(0,2,3,1))
        d2 = tonp(d2)
        
        for j in range(bs):
            save_res(imgs2[j], x12[j], d2[j][0], folder/"{}{}.png".format(i,j))

In [20]:
def fit(epoch, m, opt, MD1, MD2, MD3, MDcam):
    for i in range(epoch):
        print("--------------------epoch {} start:----------------------".format(i))
        losses = train(m, opt, MD1, MD2, MD3, MDcam)
        losses= [ str(loss) for loss in losses ]
        folder = Path("./tmp")
        folder.mkdir(exist_ok=True)
        file = folder / 'epoch{}.log'.format(i)
        with file.open('w') as f:
            f.write("\n".join(losses))
        save_model(m, str(folder / "epoch{}.M".format(i)))

In [21]:
TS = [0.3, 0.5, 0.75, 0.85, 1]
#TS = [0, 0.15, 0.3, 0.5, 0.75, 0.85, 1]

In [22]:
expfolder = Path('./experiment/loss/')
expfolder.mkdir(parents=True, exist_ok=True)

In [23]:
for ts in TS:
    thisfolder = expfolder / "tscale_{}".format(ts)
    thisfolder.mkdir(exist_ok=True)
    
    m = TriDepth(get_resnet(), 1).cuda()
    set_trainable(m.depth.rn, False)
    l = Loss(scale=1, Tscale=ts).cuda()
    opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)
    print("training model {}".format(ts))
    losses = train(m, opt, MD1, MD2, MD3, MDcam, denorm)
    print("saving model")
    save_model(m, thisfolder/"model.M".format(ts))
    print("writting log")
    logger = thisfolder/"log"
    losses = [ str(loss) for loss in losses]
    with logger.open('w') as f: f.write("\n".join(losses))
    print("predicting")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

training model 0.3
0.35192063450813293	0.3248771131038666	0.027043508365750313	0.3000439405441284	0.02483316697180271
0.3600645065307617	0.3585062623023987	0.0015582559863105416	0.3341924250125885	0.02431384101510048
0.38388633728027344	0.3822813034057617	0.0016050386475399137	0.3570185899734497	0.02526271343231201
0.3016645014286041	0.29866912961006165	0.0029953818302601576	0.27712002396583557	0.021549111232161522
0.36978036165237427	0.3654106557369232	0.004369702655822039	0.33607369661331177	0.029336968436837196
0.33864855766296387	0.3339700698852539	0.004678500350564718	0.30926382541656494	0.02470623515546322
0.4026486277580261	0.39769911766052246	0.004949495662003756	0.3671920597553253	0.030507057905197144
0.28366243839263916	0.27900373935699463	0.004658697638660669	0.25963640213012695	0.01936735212802887
0.3801097273826599	0.3753916621208191	0.004718070384114981	0.3493649363517761	0.026026729494333267
0.2780485451221466	0.27373775839805603	0.004310777876526117	0.2579200863838196	0

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


training model 0.75
0.1945158839225769	0.16735365986824036	0.027162224054336548	0.10573743283748627	0.06161621958017349
0.18147175014019012	0.18025612831115723	0.0012156172888353467	0.11935464292764664	0.06090147793292999
0.18674878776073456	0.18446657061576843	0.0022822129540145397	0.12327037751674652	0.06119620054960251
0.1535634696483612	0.15117740631103516	0.002386065199971199	0.09685372561216354	0.05432368442416191
0.18708330392837524	0.1848464012145996	0.00223689922131598	0.1149747222661972	0.06987167149782181
0.1651679128408432	0.1632000207901001	0.001967892749235034	0.10466019064188004	0.058539822697639465
0.21175414323806763	0.20956404507160187	0.002190098399296403	0.13210733234882355	0.07745671272277832
0.1392499953508377	0.1371884047985077	0.0020615835674107075	0.0903078094124794	0.04688058793544769
0.18768532574176788	0.1855563372373581	0.0021289943251758814	0.12225377559661865	0.06330256164073944
0.1254098117351532	0.1234508752822876	0.0019589378498494625	0.087662428617477

for ts in TS:
    thisfolder = expfolder / "tscale_{}".format(ts)
    thisfolder.mkdir(exist_ok=True)
    m = TriDepth(get_base(), 1).cuda()
    load_model(m, thisfolder/"model.M".format(ts))
    l = Loss(scale=1, Tscale=ts).cuda()
    print("predicting")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
m.eval()


In [ ]:
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        

In [ ]:
d2s[0].min(), d2s[0].max()